In [24]:
import requests
import zipfile
import io
import os 
import shutil
import pandas as pd
import mariadb
import pymysql
import logging
import sshtunnel


from pathlib import Path
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

ya_api = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={}'
ya_link = 'https://disk.yandex.ru/d/sVgz0GiqjhCsNg'

In [17]:
try:
    Path.mkdir(Path.cwd()/'saved_csv')
except:
    pass

save_dir = (Path.cwd()/'saved_csv').as_posix()
list_file_exist = os.listdir(save_dir)

In [18]:
list_file_exist = []

In [3]:
def get_direct_link(sharing_link):
    pk_request = requests.get(ya_api.format(sharing_link))
    # None если не удается преоброазовать ссылку
    return pk_request.json().get('href')

def download_yadisk_files(sharing_link):
    direct_link = get_direct_link(sharing_link)
    if direct_link:
        download = requests.get(direct_link)
        zips = zipfile.ZipFile(io.BytesIO(download.content))
        cnt = 0
        for member in zips.namelist():
            filename = os.path.basename(member)
            if not filename or filename in list_file_exist:
                continue
            src = zips.open(member)
            target = open(os.path.join(save_dir, filename), 'wb')
            with src, target:
                shutil.copyfileobj(src, target)
                cnt += 1
            target.close()
        print('Succesfully downloaded {} files from "{}"'.format(cnt, sharing_link))
    else:
        print('Failed to download files from "{}"'.format(sharing_link))

In [4]:
download_yadisk_files(ya_link)

Succesfully downloaded 9 files from "https://disk.yandex.ru/d/sVgz0GiqjhCsNg"


In [19]:
p = Path(save_dir).glob('**/*')
saved_files = [x.as_posix() for x in p if x.is_file() and x.name not in list_file_exist]

In [20]:
saved_files

['C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-2023.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/08-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02-23 (без дублей) 2.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/10-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/11-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/12-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/13-02-23 (без дублей).csv']

In [21]:
for f_ind in range(len(saved_files)):
    if f_ind == 0:
        saved_files_csv = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
    else:
        saved_files_csv = saved_files_csv.append(pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8'))
    saved_files_csv.reset_index(drop=True, inplace=True)

In [22]:
saved_files_csv.head(5)

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
0,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,"2-к. квартира, 44,1 м², 2/10 эт.",2-к. квартира,"44,1 м²",2/10 эт.,2641000,59887,NaN,NaN,"ул. им. Генерала Костицына, д. 11",1 день назад,Застройщик,"ООО ""СЗ Атлант-Строй""",https://www.avito.ru/user/58f18e333b0e5cf1677e...,NaN
1,https://www.avito.ru/chelyabinsk/kvartiry/1-k....,"1-к. квартира, 41 м², 1/10 эт.",1-к. квартира,41 м²,1/10 эт.,2850000,69512,NaN,р-н Курчатовский,"ул. Бейвеля, 73",1 час назад,Агентство,АН Ника-Дом,https://www.avito.ru/user/5d5a6a6aecbf4100c49f...,NaN
2,https://www.avito.ru/chelyabinsk/kvartiry/3-k....,"3-к. квартира, 60,8 м², 9/11 эт.",3-к. квартира,"60,8 м²",9/11 эт.,3675238,60448,NaN,NaN,"пос. Западный, д. 15",7 часов назад,Застройщик,"Экорайон ""Вишневая горка""",https://www.avito.ru/user/e033ce494357a96bc703...,NaN
3,https://www.avito.ru/chelyabinsk/kvartiry/3-k....,"3-к. квартира, 66 м², 6/10 эт.",3-к. квартира,66 м²,6/10 эт.,4400000,66667,NaN,р-н Калининский,"ул. 250-летия Челябинска, 9",5 дней назад,Частное лицо,NaN,NaN,NaN
4,https://www.avito.ru/chelyabinsk/kvartiry/1-k....,"1-к. квартира, 28,7 м², 3/11 эт.",1-к. квартира,"28,7 м²",3/11 эт.,1622586,56536,NaN,NaN,"пос. Западный, д. 17",7 часов назад,Застройщик,"Экорайон ""Вишневая горка""",https://www.avito.ru/user/e033ce494357a96bc703...,NaN


In [7]:
ssh_host = '92.53.64.66'
ssh_username = 'root'
ssh_password = '6hv66fhY3JF4HfwJ'
database_username = 'app_nedvizhk_usr'
database_password = 'n8vBDh6NDOKylwDn'
database_name = 'app_nedvizhka_db'
localhost = '127.0.0.1'

In [8]:
def open_ssh_tunnel(verbose=False):

    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
#     global tunnel
    tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_password = ssh_password,
        remote_bind_address = ('127.0.0.1', 3306)
    )
    
    tunnel.start()
    return tunnel

In [9]:
def mysql_connect(tunnel):
    
#     global connection
    
    connection = pymysql.connect(
        host=localhost,
        user=database_username,
        passwd=database_password,
        db=database_name,
        port=tunnel.local_bind_port
    )
    
    return connection

In [10]:
def run_query(sql):
    
    return pd.read_sql_query(sql, connection)

In [11]:
def mysql_disconnect(connection):
    
    connection.close()

In [12]:
def close_ssh_tunnel(tunnel):
    
    tunnel.close

In [13]:
tunnel = open_ssh_tunnel(verbose=False)

In [14]:
connection = mysql_connect(tunnel)

In [15]:
cursor = connection.cursor()

In [16]:
cursor.execute("""DROP table test_avito_parsed""")

0

In [25]:
ssh_host = '92.53.64.66'
ssh_username = 'root'
ssh_password = '6hv66fhY3JF4HfwJ'
database_username = 'app_nedvizhk_usr'
database_password = 'n8vBDh6NDOKylwDn'
database_name = 'app_nedvizhka_db'
localhost = '127.0.0.1'

In [26]:
server = SSHTunnelForwarder(
    ('92.53.64.66', 22),
    ssh_username="root",
    ssh_password="6hv66fhY3JF4HfwJ",
    remote_bind_address=('127.0.0.1', 3306)
    )

server.start()
local_port = str(server.local_bind_port)
engine = create_engine('mariadb+pymysql://{}:{}@{}:{}/{}'.format("app_nedvizhk_usr", "n8vBDh6NDOKylwDn", "127.0.0.1", local_port, "app_nedvizhka_db"))

In [23]:
server.stop()

In [2]:
import pandas as pd
test_df = pd.DataFrame([[1, 2, 3]], columns=['as', 'ad', 'asd'])

In [3]:
test_df

,as,ad,asd
0,1,2,3


In [27]:
saved_files_csv.to_sql(name='test_avito_parsed', con=engine, if_exists='append', chunksize=5000, method='multi')

207270